In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Heart_sound_classification-master/Heart_sound_classification-master

/content/drive/MyDrive/Heart_sound_classification-master/Heart_sound_classification-master


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Reusing the read_variable_length_csv function
def read_variable_length_csv(file_path):
    data_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line_data = line.strip().split(',')
            data_list.append(line_data)
    # Infer the maximum number of features
    max_features = max(len(line_data) for line_data in data_list) - 2  # Subtract SampleName and Label
    column_names = ['SampleName', 'Label'] + [f'Feature{i}' for i in range(1, max_features + 1)]
    df = pd.DataFrame(data_list, columns=column_names)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df

# Read data
csv_file_path = '/content/drive/MyDrive/Heart_sound_classification-master/Heart_sound_classification-master/csv/training-b.csv'
data = read_variable_length_csv(csv_file_path)

# Replace 'None' with zeros and convert features to floats
features = data.iloc[:, 2:].fillna(0).astype(float)

# Flatten the features since SVM does not need the data to be padded as in sequential models
features_flat = features.values

# Encode the labels
labels = LabelEncoder().fit_transform(data.iloc[:, 1].values)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features_flat, labels, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create SVM model
svm_model = SVC(kernel='rbf', C=1.0, random_state=42)

# Train the SVM model
svm_model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of SVM model: {accuracy:.4f}')


Accuracy of SVM model: 0.6951


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def read_variable_length_csv(file_path):
    data_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line_data = line.strip().split(',')
            data_list.append(line_data)
    max_features = max(len(line_data) for line_data in data_list) - 2  # Subtract SampleName and Label
    column_names = ['SampleName', 'Label'] + [f'Feature{i}' for i in range(1, max_features + 1)]
    df = pd.DataFrame(data_list, columns=column_names)
    # Randomly shuffle the dataframe
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df

# Path to your CSV file
csv_file_path = '/content/drive/MyDrive/Heart_sound_classification-master/Heart_sound_classification-master/csv/training-b.csv'
data = read_variable_length_csv(csv_file_path)

# Replace 'None' with zeros and convert features to floats
features = data.iloc[:, 2:].fillna(0).astype(float).values

# Encode the labels
labels = LabelEncoder().fit_transform(data.iloc[:, 1].values)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=43)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=43)
xgb_model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test_scaled)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of XGBoost model: {accuracy:.4f}')


Accuracy of XGBoost model: 0.7317


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

def read_variable_length_csv(file_path):
    data_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line_data = line.strip().split(',')
            data_list.append(line_data)
    max_features = max(len(line_data) for line_data in data_list) - 2  # Subtract SampleName and Label
    column_names = ['SampleName', 'Label'] + [f'Feature{i}' for i in range(1, max_features + 1)]
    df = pd.DataFrame(data_list, columns=column_names)
    # Randomly shuffle the dataframe
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df

# Path to your CSV file
csv_file_path = '/content/drive/MyDrive/Heart_sound_classification-master/Heart_sound_classification-master/csv/training-b.csv'
data = read_variable_length_csv(csv_file_path)

# Replace 'None' with zeros and convert features to floats
features = data.iloc[:, 2:].fillna(0).astype(float).values

# Encode the labels
labels = LabelEncoder().fit_transform(data.iloc[:, 1].values)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=4)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model architecture
model = Sequential([
    Dense(1024, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
y_pred = (y_pred > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of MLP model: {accuracy:.4f}')


Epoch 1/50
9/9 [==============================] - 2s 40ms/step - loss: 3.6869 - accuracy: 0.5364 - val_loss: 0.8864 - val_accuracy: 0.7121
Epoch 2/50
9/9 [==============================] - 0s 18ms/step - loss: 6.3167 - accuracy: 0.5594 - val_loss: 0.7441 - val_accuracy: 0.7121
Epoch 3/50
9/9 [==============================] - 0s 18ms/step - loss: 7.1195 - accuracy: 0.6092 - val_loss: 0.6345 - val_accuracy: 0.7121
Epoch 4/50
9/9 [==============================] - 0s 18ms/step - loss: 4.9808 - accuracy: 0.6513 - val_loss: 0.5608 - val_accuracy: 0.7121
Epoch 5/50
9/9 [==============================] - 0s 16ms/step - loss: 6.9624 - accuracy: 0.6322 - val_loss: 0.5486 - val_accuracy: 0.7121
Epoch 6/50
9/9 [==============================] - 0s 16ms/step - loss: 3.1022 - accuracy: 0.6322 - val_loss: 0.5543 - val_accuracy: 0.7121
Epoch 7/50
9/9 [==============================] - 0s 17ms/step - loss: 3.9894 - accuracy: 0.6322 - val_loss: 0.5489 - val_accuracy: 0.7121
Epoch 8/50
9/9 [===========

3/3 [==============================] - 0s 5ms/step
Accuracy of MLP model: 0.6707
